# Inverted Bottleneck (MBConv) — Teori ve Uygulama

Bu defterde **Inverted Bottleneck / MBConv** yapısını ayrıntılı inceleyeceksin:

- Hangi problemden ortaya çıktı?
- Neden "inverted" (tersine çevrilmiş) bottleneck deniyor?
- Blok yapısı: `1×1 expand → 3×3 depthwise → 1×1 project (+ residual)`
- Neden son katmanda `linear` (aktivasyonsuz) kullanılıyor?
- PyTorch ile MBConv bloğunu nasıl kodlarız?
- Standart bir `Conv2d` bloğu ile parametre / FLOPs açısından farkı nedir?

Hedef, bu bloğu **MobileNetV2 / EfficientNet gibi mimarilerde** gördüğünde,
sadece "ezbere" değil, **tasarım mantığıyla birlikte** anlayabilmektir.


## 1. Klasik Bottleneck (ResNet) ve Problem

Önce klasik ResNet bottleneck yapısını hatırlayalım:

- Girdi kanalı: `C_in`
- Çıkış kanalı: `C_out`
- Ara (bottleneck) kanal: `C_mid` (genelde `C_mid = C_out / 4`)

Blok yapısı kabaca şöyledir:

1. **1×1 conv**: `C_in → C_mid`  (kanal sayısını **daraltır**)
2. **3×3 conv**: `C_mid → C_mid`
3. **1×1 conv**: `C_mid → C_out` (kanal sayısını **geri büyütür**)
4. Gerekirse residual bağlantı: `x + F(x)`

Bu yapıya *bottleneck* denmesinin sebebi, **orta katmanda kanal sayısının daraltılmasıdır**.

### Problem: Mobil / Edge Ortamlar

ResNet gibi mimariler:

- Yüksek doğruluk sağlar,
- Fakat FLOPs ve parametre sayısı **mobil cihazlar için çok yüksektir**.

Ayrıca yapılan analizler şunu gösterir:

- Özellikle düşük kanal sayılarında,
- Ortadaki dar boğaz (bottleneck) **bilgi kaybına ve temsil gücü düşüşüne** sebep olabilir.

Bu noktada **MobileNetV2**'nin fikri devreye girer.


## 2. Inverted Bottleneck Fikri (MobileNetV2 / MBConv)

MobileNetV2, şu soruyu sorar:

> "Neden önce daraltıyoruz? Özellikle düşük çözünürlükte zaten az bilgi varken,
>  kanalı daha da daraltmak temsil gücünü azaltmıyor mu?"

Cevap olarak **bottleneck mantığını tersine çevirir**:

1. Girişte **kanalı genişlet** (expand)
2. Orta katmanda **uzamsal işlemi depthwise ile yap**
3. Sonda **kanalı tekrar daralt** (project)
4. Mümkünse residual ekle

Yani:

- ResNet bottleneck:  
  `1×1 reduce → 3×3 conv → 1×1 expand`

- Inverted bottleneck (MBConv):  
  `1×1 expand → 3×3 depthwise → 1×1 project`

Bu yüzden adı **"inverted bottleneck"**,
çünkü daraltma ve genişletme sırası **tersine çevrilmiştir**.


## 3. MBConv Bloğunun Yapısı

Bir MBConv bloğu tipik olarak şu adımlara sahiptir:

1. **1×1 Conv (Expansion / Genişletme)**  
   - Giriş kanalı: `C_in`  
   - Genişletilmiş kanal: `C_exp = t · C_in` (t: expansion factor, örn. 4 veya 6)  
   - Aktivasyon: ReLU6 veya Swish/SiLU

2. **3×3 Depthwise Conv (Uzamsal İşlem)**  
   - Girdi/Çıktı kanal: `C_exp`  
   - `groups = C_exp` → her kanal kendi filtresiyle işlenir  
   - Aktivasyon: ReLU6 / Swish

3. **1×1 Conv (Projection / Daraltma)**  
   - Giriş: `C_exp`  
   - Çıkış: `C_out` (genelde `C_out = C_in`)  
   - **Lineer** (çoğunlukla aktivasyon yok)

4. **Residual Bağlantı (opsiyonel)**  
   - Eğer `stride = 1` ve `C_in == C_out` ise:  
     `y = x + F(x)`

Bu blok MobileNetV2 ve EfficientNet'te temel yapı taşıdır.


## 4. Neden Son Katmanda Aktivasyon Yok? (Linear Bottleneck)

MobileNetV2 makalesinin kritik noktalarından biri de şudur:

> Yüksek boyutlu **düşük-enerjili manifoldlar** üzerinde çalışırken,
> son projeksiyon katmanında ReLU gibi non-lineer aktivasyon kullanmak,
> bilgiyi gereksiz yere "sıkıştırarak" kayba yol açabilir.

Bu nedenle:

- Genişletilmiş kanalda (C_exp) ReLU/Swish gibi aktivasyonlar kullanılır.
- Son 1×1 "projection" katmanında **aktivasyon kullanılmaz**.
- Böylece bilgi **lineer bir uzaya projekte edilir**, gereksiz clipping engellenir.

Bu fikre **"linear bottleneck"** denir.


In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class MBConv(nn.Module):
    """
    Inverted Bottleneck (MBConv) bloğu.
    
    Yapı:
    - 1x1 expansion conv (C_in -> C_exp)
    - 3x3 depthwise conv (C_exp -> C_exp, groups=C_exp)
    - 1x1 projection conv (C_exp -> C_out)
    - opsiyonel residual (stride=1 ve C_in==C_out ise)
    """
    def __init__(self, c_in, c_out, expansion_factor=6, stride=1, kernel_size=3):
        super().__init__()
        self.stride = stride
        self.use_residual = (stride == 1 and c_in == c_out)
        
        c_exp = c_in * expansion_factor
        
        # 1) Expansion 1x1 conv
        self.expand = nn.Sequential(
            nn.Conv2d(c_in, c_exp, kernel_size=1, bias=False),
            nn.BatchNorm2d(c_exp),
            nn.SiLU(inplace=True)  # Swish benzeri; ReLU6 da kullanılabilir
        )
        
        # 2) Depthwise 3x3 conv
        padding = kernel_size // 2
        self.depthwise = nn.Sequential(
            nn.Conv2d(
                c_exp,
                c_exp,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
                groups=c_exp,   # depthwise
                bias=False
            ),
            nn.BatchNorm2d(c_exp),
            nn.SiLU(inplace=True)
        )
        
        # 3) Projection 1x1 conv (linear)
        self.project = nn.Sequential(
            nn.Conv2d(c_exp, c_out, kernel_size=1, bias=False),
            nn.BatchNorm2d(c_out)
            # Aktivasyon YOK (linear bottleneck)
        )
    
    def forward(self, x):
        identity = x
        
        out = self.expand(x)
        out = self.depthwise(out)
        out = self.project(out)
        
        if self.use_residual:
            out = out + identity
        
        return out

# Küçük bir test:
x = torch.randn(1, 16, 32, 32)
block = MBConv(c_in=16, c_out=16, expansion_factor=6, stride=1)
y = block(x)
print("Girdi şekli :", x.shape)
print("Çıktı şekli :", y.shape)


Girdi şekli : torch.Size([1, 16, 32, 32])
Çıktı şekli : torch.Size([1, 16, 32, 32])


## 5. Standart Conv Bloğu ile Parametre Karşılaştırması

Şimdi aynı giriş/çıkış kanal sayıları için:

- Basit bir "standart" konv blok (Conv-BN-ReLU-Conv-BN-ReLU)
- MBConv bloğu

arasındaki parametre sayısını karşılaştıralım.


In [2]:
def count_params(m):
    return sum(p.numel() for p in m.parameters() if p.requires_grad)

class SimpleConvBlock(nn.Module):
    """
    Karşılaştırma için basit bir blok:
    Conv3x3 -> BN -> ReLU -> Conv3x3 -> BN -> ReLU
    """
    def __init__(self, c_in, c_out):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(c_in, c_out, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(c_out),
            nn.ReLU(inplace=True),
            nn.Conv2d(c_out, c_out, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(c_out),
            nn.ReLU(inplace=True),
        )
    
    def forward(self, x):
        return self.conv(x)

c_in, c_out = 16, 16

simple_block = SimpleConvBlock(c_in, c_out)
mbconv_block = MBConv(c_in, c_out, expansion_factor=6, stride=1)

print("SimpleConvBlock parametre sayısı:", count_params(simple_block))
print("MBConv parametre sayısı        :", count_params(mbconv_block))

x = torch.randn(1, c_in, 32, 32)
y1 = simple_block(x)
y2 = mbconv_block(x)
print("SimpleConvBlock çıktı şekli:", y1.shape)
print("MBConv çıktı şekli       :", y2.shape)


SimpleConvBlock parametre sayısı: 4672
MBConv parametre sayısı        : 4352
SimpleConvBlock çıktı şekli: torch.Size([1, 16, 32, 32])
MBConv çıktı şekli       : torch.Size([1, 16, 32, 32])


## 6. MBConv Nerede Kullanılır?

**Başlıca kullanım alanları:**

- **MobileNetV2**: MBConv, mimarinin ana yapı taşıdır.
- **EfficientNet**: MBConv + Squeeze-and-Excitation (SE) birleştirilmiştir.
- Birçok mobil/edge odaklı CNN mimarisi:  
  - Düşük FLOPs  
  - Yüksek temsil gücü  
  - Yüksek doğruluk / maliyet dengesi

**Ne zaman tercih edilir?**

- Hesaplama bütçesi kısıtlıysa (mobil, gömülü sistemler),
- Parametre sayısını azaltmak istiyorsan,
- Buna rağmen klasik Conv bloklarına yakın veya daha iyi doğruluk hedefliyorsan,
- Özellikle 3×3 conv maliyetini düşürmek istiyorsan (depthwise kullanımı).

Özetle MBConv:

- Temsili önce **yüksek boyuta çıkarır** (expand),
- Uzamsal işlemi **depthwise ile ucuz yapar**,
- Sonra temsili **daha kompakt boyuta geri projekte eder** (project),
- Ve bunu **gereksiz non-lineer sıkıştırma olmadan** (linear bottleneck) yapar.


## 7. Kısa Özet

- Inverted bottleneck, klasik ResNet bottleneck'in tersine çevrilmiş halidir.
- Blok yapısı: **1×1 expand → 3×3 depthwise → 1×1 project (+ residual)**.
- Amaç:
  - Mobil/edge ortamlarda **yüksek verimlilik**,
  - Yüksek boyutlu temsilde non-lineer dönüşümleri tutup,
  - Son projeksiyonda **lineer** dönüşüm ile bilgi kaybını azaltmak.
- MBConv, MobileNetV2 ve EfficientNet gibi modern mimarilerin bel kemiğidir.
